In [160]:
class Operad:
    def __init__(self):
        self.colors = {}
        self.operations = {}

    def add_color(self, name, prop={}):
        if t := self.colors.get(name):
            self.colors[name] = (t, prop)
        else:
            self.colors[name] = prop
        return name, t

    def add_operation(self, name, prop={}):
        self.operations[name] = prop
        return name


class Tree:
    def __init__(self, operation, trunk, branches, operad):

        trunk, parent = operad.add_color(trunk, self)
        self.trunk = trunk
        if parent:
            parent.branches[trunk] = self
            self.depth = parent.depth + 1
        else:
            self.depth = 0
        self.branches = {
            operad.add_color(branch, self)[0]: uTree(branch, depth=self.depth)
            for branch in branches
        }
        self.node = operad.add_operation(operation, self)

    def print_edges(self):
        return "".join(
            [f"{str(self.trunk)}\n"]
            + [
                "\t" * (self.depth + 1) + f"{branch.print_edges()}\n"
                for branch in self.branches.values()
            ]
        )

    def print_nodes(self):
        return "".join(
            [f"{str(self.node)}\n"]
            + [
                "\t" * (self.depth + 1) + f"{branch.print_nodes()}\n"
                for branch in self.branches.values()
            ]
        )

    def __str__(self):
        return f"{self.node}({','.join(self.branches.keys())};{self.trunk})"

    def __repr__(self):
        return f"{self.node}({','.join(self.branches.keys())};{self.trunk})"


class uTree(Tree):
    def __init__(self, name, depth):
        self.trunk = name
        self.branches = {}
        self.node = None
        self.depth = depth


In [161]:
import re
from functools import cmp_to_key
from copy import deepcopy
from collections import defaultdict

def string_to_tree_space(string, operad):

    operads = []
    operations = string.split("|")
    operations = sorted(operations, key=cmp_to_key(compare))
    regex = r"(.*)\((.*)\)"
    for operation in operations:
        match = re.match(regex, operation)
        if not match:
            raise RuntimeError(f"Operation not defined correctly {operation}")
        operation, parameters = match.groups()
        branches, trunk = parameters.split(";")
        operads.append(Tree(operation, trunk, branches.split(","), operad))
    return  operads[0], operad


def tree_space_to_string(tree_space):
    _, operad = tree_space
    return "|".join(str(tree) for tree in operad.operations.values())

def _recursive_str(tree, s):
    s.append(str(tree))
    for branch in tree.branches.values():
        if not isinstance(branch, uTree):
            _recursive_str(branch, s)
    return s

def tree_to_string(tree):
    operations = sorted(_recursive_str(tree, []), key=cmp_to_key(compare))
    return "|".join(operations)

def extract_compare(item):
    regex = r"(.*)\((.*)\)"
    match = re.match(regex, item)
    if not match:
        raise RuntimeError(f"Operation not defined correctly {item}")
    _, parameters = match.groups()
    branches, trunk = parameters.split(";")
    return trunk, branches.split(",")
    
def compare(item1, item2):
    item1 = extract_compare(item1)
    item2 = extract_compare(item2)
    
    if not item1 or not item2:
        return 0
    if item1[0] in item2[1]:
        return 1
    elif item2[0] in item1[1]:
        return -1
    else:
        return 0

def bubbleSort(arr):
    n = len(arr)
    items = [(extract_compare(e), e) for e in arr]
    final = []
    for _ in range(n):
        aux = items[0]
        j = 0
        for i, item in enumerate(items):
            if aux[0][0] in item[0][1]:
                aux = item
                j=i
        items.pop(j)
        final.append(aux[1])
    return final


# S = string_to_tree_space("o0w(1,2;0)", Operad())
# T = string_to_tree_space("o0b(b,c;a)|o1b(d;b)", Operad())
# S = string_to_tree_space("0W(1;0)|1W(2,3;1)", Operad())
# T = string_to_tree_space("0B(b,d;a)|1B(c;b)", Operad())
# S = string_to_tree_space("0W(1;0)|1W(2,3;1)", Operad())
# T = string_to_tree_space("0B(b,d;a)|1B(c;b)|2B(e;d)", Operad())
# S = string_to_tree_space("o0w(1;0)", Operad())
# T = string_to_tree_space("o0b(b;a)", Operad())
# T = string_to_tree_space("o0w(c2,c1;c0)|o1w(c3,c4;c1)|o2w(c5,c6;c2)", Operad())
# S = string_to_tree_space("o0b(d2,d1;d0)|o1b(d3,d4;d1)|o2b(d5,d6;d2)", Operad())
# S = string_to_tree_space("W(b,c;a)", Operad())
# T = string_to_tree_space("B(1;0)", Operad())
# S = string_to_tree_space("0W(1;0)|1W(2,3;1)", Operad())
# T = string_to_tree_space("0B(b,d;a)|1B(c;b)|2B(e;d)", Operad())
S = string_to_tree_space("0W(1;0)|1W(2,4,6;1)|2W(3;2)|3W(5;4)|4W(7;6)", Operad())
T = string_to_tree_space("0B(b,f;a)|1B(c;b)|2B(d;c)|3B(e;d)|4B(g;f)", Operad())
# S = string_to_tree_space("0W(1;0)|1W(2,3;1)|2W(;3)", Operad())
# T = string_to_tree_space("0B(b,c;a)|1B(;c)", Operad())
# S = string_to_tree_space("0W(1;0)|1W(2,3;1)", Operad())
# T = string_to_tree_space("0B(b,c;a)", Operad())

# print(tree_space_to_string(S))
# print(tree_to_string(S[0]))
# print(tree_space_to_string(T))
# print(tree_to_string(T[0]))
# print(S[0].print_nodes())

In [162]:
class TreeMerger:
    def __init__(self, S, T):
        self.S_og_space = S
        self.T_og_space = T
        self.tree_str = self.merge(deepcopy(S[0]),deepcopy(T[0]))
        
    def get_result(self):
        return self.tree_str, self.S_og_space[1].operations, self.T_og_space[1].operations

    def merge(self, S, T):
        self.add_color_base(S, T.trunk)        
        return tree_to_string(S)
        
    def add_color_edge(self, T, c, depth):
        T.trunk = f"{c}-{T.trunk}"
        T.branches = self.rename_keys(T, c, True)
        T.depth += depth
        for Ti in T.branches.values():
            self.add_color_edge(Ti, c, depth)
        return T

    def rename_keys(self, S, c, inv=False):
        aux = {}
        for i, Si in S.branches.items():
            if not inv:
                aux[f"{i}-{c}"] = Si
            else:
                aux[f"{c}-{i}"] = Si
        return aux

    def add_color_base(self, S, c):
        S.trunk = f"{S.trunk}-{c}"
        S.branches = self.rename_keys(S, c)
        for i, Si in S.branches.items():
            if isinstance(Si, uTree):
                S.branches[i] = self.add_color_edge(
                    deepcopy(self.T_og_space[0]), i.split("-")[0], S.depth + 1
                )
            else:
                self.add_color_base(Si, c)
        
s = TreeMerger(deepcopy(S), deepcopy(T))
result = s.get_result()
print(result)

('0W(1-a;0-a)|1W(2-a,3-a;1-a)|0B(2-b,2-d;2-a)|1B(2-c;2-b)|2B(2-e;2-d)|0B(3-b,3-d;3-a)|1B(3-c;3-b)|2B(3-e;3-d)', {'0W': 0W(1;0), '1W': 1W(2,3;1)}, {'0B': 0B(b,d;a), '1B': 1B(c;b), '2B': 2B(e;d)})


In [163]:
class TreeManipulator:
    def __init__(self, tree_str, S_og_operations, T_og_operations, operad):
        self.tree_str = tree_str
        self.S_og_operations = S_og_operations
        self.T_og_operations = T_og_operations
        self.tree, self.operad = string_to_tree_space(tree_str, operad)

    def find_percolant_branches(self, S=None, found=[]):
        if not S:
            S = self.tree
        if S.node in self.S_og_operations.keys():
            if any(
                branch.node in self.T_og_operations.keys()
                for branch in S.branches.values()
            ):
                found.append(S)

        for Si in S.branches.values():
            self.find_percolant_branches(Si, found)
        return found

    def make_percolation(self, location):
        operations = self.tree_str.split("|")
        branches = list(location.branches.values())
        new_node = branches[0].node
        old_node = location.node
        to_change = [str(location)] + [str(branch) for branch in branches]
        old_operation = self.S_og_operations[old_node]
        new_operation = self.T_og_operations[new_node]

        morfed = []
        for _, c in new_operation.branches.items():
            morfed.append(
                f"{old_operation.node}({','.join(map(lambda x: x+'-'+c.trunk, old_operation.branches.keys()))};{old_operation.trunk}-{c.trunk})"
            )
        morfed.append(
            f"{new_operation.node}({','.join(map(lambda x: old_operation.trunk+'-'+x, new_operation.branches.keys()))};{old_operation.trunk}-{new_operation.trunk})"
        )

        for branch in to_change:
            operations.remove(branch)
        operations += morfed
        operations = bubbleSort(operations)
        new_tree = "|".join(operations)
        return new_tree


tm = TreeManipulator(*result, Operad())
print(tm.find_percolant_branches())
tm.make_percolation(tm.find_percolant_branches()[0])


[1W(2-a,3-a;1-a)]


'0W(1-a;0-a)|0B(1-b,1-d;1-a)|1W(2-b,3-b;1-b)|1B(2-c;2-b)|1W(2-d,3-d;1-d)|2B(2-e;2-d)|1B(3-c;3-b)|2B(3-e;3-d)'

In [166]:
class ShuffleLattice:
    def __init__(self, S, T):
        self.tm = TreeMerger(deepcopy(S), deepcopy(T))
        self.initial_tree, *self.operations = self.tm.get_result()
        self.skeleton = defaultdict(list)
        self.initialize_skeleton()
        self.generate_shuffles()
        
    def initialize_skeleton(self):
        self.skeleton[self.initial_tree] = []

    def generate_shuffles(self):
        queue = []
        queue.append(self.initial_tree)

        while len(queue):
            tree = queue.pop()
            manipulator = TreeManipulator(tree, *self.operations, Operad())

            for location in  manipulator.find_percolant_branches(found=[]):
                fingerprint = manipulator.make_percolation(location)
                if not self.fingerprint_in_skeleton(fingerprint, tree):
                    queue.append(fingerprint)
                
    def fingerprint_in_skeleton(self, fingerprint, old):
        operations = set(fingerprint.split("|"))
        found = False
        for key in self.skeleton:
            if operations == set(key.split("|")):
                self.skeleton[key].append(fingerprint)
                found = True
        if not found:
            self.skeleton[fingerprint].append(old)
        return found
    

shl = ShuffleLattice(S, T)
# print([len(a) for a in shl.skeleton.values()])
# print("\n".join(shl.skeleton.keys()))
print("Number of trees:", len(shl.skeleton))
# print(shl.skeleton)
for i, tree in enumerate(shl.skeleton.keys()):
    print(f"R{i}")
    space = string_to_tree_space(tree, Operad())
    print(space[0].print_nodes())

Number of trees: 14
R0
0W
	1W
		0B
			1B
				None


			2B
				None



		0B
			1B
				None


			2B
				None





R1
0W
	0B
		1W
			1B
				None


			1B
				None



		1W
			2B
				None


			2B
				None





R2
0B
	0W
		1W
			1B
				None


			1B
				None




	0W
		1W
			2B
				None


			2B
				None





R3
0W
	0B
		1B
			1W
				None

				None



		1W
			2B
				None


			2B
				None





R4
0W
	0B
		1W
			1B
				None


			1B
				None



		2B
			1W
				None

				None





R5
0B
	0W
		1W
			1B
				None


			1B
				None




	0W
		2B
			1W
				None

				None





R6
0W
	0B
		1B
			1W
				None

				None



		2B
			1W
				None

				None





R7
0B
	0W
		1B
			1W
				None

				None




	0W
		2B
			1W
				None

				None





R8
0B
	1B
		0W
			1W
				None

				None




	0W
		2B
			1W
				None

				None





R9
0B
	0W
		1B
			1W
				None

				None




	2B
		0W
			1W
				None

				None





R10
0B
	1B
		0W
			1W
				None

				None




	2B
		0W
			1W
				None

				None





R11
0B
	0W
		1W
			1B
				No